In [2]:
import os
import cv2
import numpy as np
import networkx as nx
import torch
import torchvision.transforms as transforms
from skimage.segmentation import felzenszwalb
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.color import rgb2gray
from tqdm import tqdm

In [5]:
# Define dataset path
DATASET_PATH = r"C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\Dataset" 
CATEGORIES = ["Negative", "Positive"]

# Image preprocessing
IMG_SIZE = 256  # Resize images

# Function to extract texture features using GLCM (Gray-Level Co-occurrence Matrix)
def extract_texture_features(gray_image):

    gray_image = (gray_image * 255).astype(np.uint8)
    
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = graycomatrix(gray_image, distances=distances, angles=angles, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast').mean()
    dissimilarity = graycoprops(glcm, 'dissimilarity').mean()
    homogeneity = graycoprops(glcm, 'homogeneity').mean()
    energy = graycoprops(glcm, 'energy').mean()
    correlation = graycoprops(glcm, 'correlation').mean()
    return [contrast, dissimilarity, homogeneity, energy, correlation]

# Function to process image and create a graph
def image_to_graph(image_path):
    # Load image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

    # Apply graph-based segmentation
    segments = felzenszwalb(image, scale=100, sigma=0.5, min_size=50)

    # Convert image to grayscale for texture extraction
    gray_image = rgb2gray(image)

    # Create a graph
    G = nx.Graph()

    # Get unique segments (superpixels)
    unique_segments = np.unique(segments)

    for seg_id in unique_segments:
        # Get segment mask
        mask = segments == seg_id
        y, x = np.where(mask)

        if len(y) == 0:  # Skip empty segments
            continue

        # Compute node features (color & texture)
        mean_color = np.mean(image[y, x], axis=0)  # RGB mean color
        texture_features = extract_texture_features(gray_image)  # Texture properties

        # Add node to the graph
        G.add_node(seg_id, color=mean_color.tolist(), texture=texture_features)

    # Add edges based on adjacency of segments
    for y in range(segments.shape[0] - 1):
        for x in range(segments.shape[1] - 1):
            seg1 = segments[y, x]
            seg2 = segments[y + 1, x]
            seg3 = segments[y, x + 1]

            if seg1 != seg2:
                G.add_edge(seg1, seg2)

            if seg1 != seg3:
                G.add_edge(seg1, seg3)

    return G



In [7]:
graphs = []
labels = []

for category in CATEGORIES:
    label = 1 if category == "Positive" else 0
    folder_path = os.path.join(DATASET_PATH, category)

    print(f"Processing {category} images...")
    for i, img_name in enumerate(tqdm(os.listdir(folder_path))):
        if i >= 100:  # Limit to first 100 images
            break
        img_path = os.path.join(folder_path, img_name)
        graph = image_to_graph(img_path)
        graphs.append(graph)
        labels.append(label)

# Save dataset
torch.save({"graphs": graphs, "labels": labels}, "graph_dataset.pt")
print("Graph dataset saved successfully!")


Processing Negative images...


  0%|          | 0/20000 [00:00<?, ?it/s]

  0%|          | 100/20000 [05:53<19:32:59,  3.54s/it]


Processing Positive images...


  0%|          | 100/20000 [06:18<20:54:37,  3.78s/it]


Graph dataset saved successfully!


In [12]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index  # Node features & adjacency
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)  # Classification output

# Example initialization
model = GCN(input_dim=16, hidden_dim=32, output_dim=2)  # Assuming 16 node features
print(model)


GCN(
  (conv1): GCNConv(16, 32)
  (conv2): GCNConv(32, 2)
)


In [14]:
import torch.optim as optim
from torch_geometric.data import DataLoader

# Load dataset (assuming PyG format)
dataset = torch.load("graph_dataset.pt", weights_only=False)
graphs, labels = dataset["graphs"], dataset["labels"]

# Convert to PyG Data format
from torch_geometric.data import Data
graph_data = []
for g, l in zip(graphs, labels):
    # Extract node features
    node_features = [g.nodes[n].get("feature", torch.zeros(16)) for n in g.nodes()]
    node_features = torch.stack(node_features)  # Stack into tensor

    # Convert edges
    edge_index = torch.tensor(list(g.edges)).t().contiguous()

    # Create PyG Data object
    data = Data(x=node_features, edge_index=edge_index, y=torch.tensor([l]))
    graph_data.append(data)


# DataLoader
train_loader = DataLoader(graph_data, batch_size=16, shuffle=True)

# Model, loss, and optimizer
model = GCN(input_dim=16, hidden_dim=32, output_dim=2)  # Adjust input_dim based on feature size
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training Loop
for epoch in range(10):
    model.train()
    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")


C:\Users\EDWIN\AppData\Roaming\Python\Python313\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


ValueError: Expected input batch_size (1034) to match target batch_size (16).